# Bank Marketing — Term Deposit Subscription Prediction

**Date:** August 24, 2025

This notebook reproduces the full analysis pipeline.

## 1. Setup

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, RocCurveDisplay
)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

DATA_PATH = "/mnt/data/bankmarketing.csv"
df = pd.read_csv(DATA_PATH)
df.head()


## 2. Target & Feature Preparation

In [ ]:

possible_targets = ['y', 'deposit', 'subscribed', 'term_deposit', 'target']
target_col = None
for c in df.columns:
    if c.lower() in possible_targets:
        target_col = c
        break
if target_col is None:
    for c in df.columns:
        if df[c].nunique() == 2:
            vals = set(str(v).lower() for v in df[c].dropna().unique())
            if vals <= {"yes","no"} or vals <= {"0","1"} or vals <= {"true","false"}:
                target_col = c
                break
if target_col is None:
    target_col = df.columns[-1]

y = df[target_col].astype(str).str.lower().replace({"yes":1,"no":0,"true":1,"false":0})
try:
    y = y.astype(int)
except Exception:
    y, _ = pd.factorize(y)

X = df.drop(columns=[target_col]).copy()
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = [c for c in X.columns if c not in numeric_features]

print("Target:", target_col)
print("Numeric:", len(numeric_features), "Categorical:", len(categorical_features))


## 3. EDA

In [ ]:

pd.Series(y).value_counts(normalize=True)


In [ ]:

plt.figure()
pd.Series(y).value_counts().sort_index().plot(kind="bar")
plt.title("Target Distribution (0 = No, 1 = Yes)")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()


In [ ]:

if len(numeric_features) > 0:
    col = 'age' if 'age' in X.columns else numeric_features[0]
    plt.figure()
    X[col].dropna().plot(kind="hist", bins=30)
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()


In [ ]:

if len(numeric_features) >= 2:
    corr = X[numeric_features].corr(numeric_only=True)
    plt.figure(figsize=(6,5))
    im = plt.imshow(corr, aspect='auto')
    plt.title("Correlation Heatmap (Numeric Features)")
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.xticks(range(len(numeric_features)), numeric_features, rotation=90)
    plt.yticks(range(len(numeric_features)), numeric_features)
    plt.tight_layout()
    plt.show()


## 4. Preprocessing & Split

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler(with_mean=False))
])
categorical_transformer = OneHotEncoder(handle_unknown="ignore", sparse=True)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ],
    remainder="drop"
)


## 5. Modeling

In [ ]:

models = {
    "LogisticRegression": LogisticRegression(max_iter=200),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
}
results = []
for name, model in models.items():
    clf = Pipeline(steps=[("preprocess", preprocessor), ("model", model)])
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    if hasattr(clf, "predict_proba"):
        y_proba = clf.predict_proba(X_test)[:,1]
    else:
        y_proba = y_pred.astype(float)
    metrics = {
        "model": name,
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "f1": f1_score(y_test, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_test, y_proba)
    }
    results.append(metrics)
    RocCurveDisplay.from_predictions(y_test, y_proba)
    plt.title(f"ROC Curve - {name}")
    plt.show()

pd.DataFrame(results).sort_values("roc_auc", ascending=False)


## 6. Best Model & Confusion Matrix

In [ ]:

results_df = pd.DataFrame(results).sort_values("roc_auc", ascending=False)
best_model_name = results_df.iloc[0]["model"]
best_model = Pipeline(steps=[("preprocess", preprocessor), ("model", models[best_model_name])])
best_model.fit(X_train, y_train)
best_pred = best_model.predict(X_test)
cm = confusion_matrix(y_test, best_pred)
cm


In [ ]:

plt.figure()
plt.imshow(cm, interpolation='nearest')
plt.title(f"Confusion Matrix - {best_model_name}")
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ["0","1"])
plt.yticks(tick_marks, ["0","1"])
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center")
plt.ylabel('True label'); plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()


## 7. Feature Importance / Coefficients

In [ ]:

def get_feature_names(preprocessor, X):
    feature_names = []
    numeric_features = preprocessor.transformers_[0][2]
    categorical_features = preprocessor.transformers_[1][2]
    feature_names.extend(numeric_features)
    ohe = preprocessor.named_transformers_["cat"]
    try:
        ohe_names = ohe.get_feature_names_out(categorical_features).tolist()
    except Exception:
        ohe_names = ohe.get_feature_names(categorical_features).tolist()
    feature_names = numeric_features + ohe_names
    return feature_names

preprocessor.fit(X_train)
feature_names = get_feature_names(preprocessor, X_train)
Xt_train = preprocessor.transform(X_train)
Xt_test = preprocessor.transform(X_test)
mdl = models[best_model_name]
mdl.fit(Xt_train, y_train)

fi_df = None
if hasattr(mdl, "feature_importances_"):
    importances = mdl.feature_importances_
    fi_df = pd.DataFrame({"feature": feature_names, "importance": importances}).sort_values("importance", ascending=False).head(20)
elif hasattr(mdl, "coef_"):
    coefs = mdl.coef_
    if getattr(coefs, "ndim", 1) > 1:
        coefs = coefs[0]
    fi_df = pd.DataFrame({"feature": feature_names, "importance": np.abs(coefs)}).sort_values("importance", ascending=False).head(20)

fi_df


In [ ]:

if fi_df is not None and not fi_df.empty:
    plt.figure(figsize=(8, max(4, 0.3 * len(fi_df))))
    plt.barh(fi_df["feature"][::-1], fi_df["importance"][::-1])
    plt.title(f"Top Features - {best_model_name}")
    plt.xlabel("Importance (abs coef or Gini importance)")
    plt.tight_layout()
    plt.show()


## 8. Conclusions & Recommendations
- Use predicted probabilities to prioritize leads.
- Tailor messaging to top-performing segments.
- Retrain periodically with new data.